## 时序的关于时间的embedding

In [2]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import math

In [3]:
class FixedEmbedding(nn.Module):
    def __init__(self, c_in, d_model):
        super(FixedEmbedding, self).__init__()

        w = torch.zeros(c_in, d_model).float()
        w.require_grad = False

        position = torch.arange(0, c_in).float().unsqueeze(1)
        div_term = (torch.arange(0, d_model, 2).float() *
                    -(math.log(10000.0) / d_model)).exp()

        w[:, 0::2] = torch.sin(position * div_term)
        w[:, 1::2] = torch.cos(position * div_term)

        self.emb = nn.Embedding(c_in, d_model)
        self.emb.weight = nn.Parameter(w, requires_grad=False)

    def forward(self, x):
        return self.emb(x).detach()

In [5]:
class TemporalEmbedding(nn.Module):
    def __init__(self, d_model, embed_type='fixed', freq='h'):
        super(TemporalEmbedding, self).__init__()

        minute_size = 4
        hour_size = 24
        weekday_size = 7
        day_size = 32
        month_size = 13

        Embed = FixedEmbedding if embed_type == 'fixed' else nn.Embedding
        if freq == 't':
            self.minute_embed = Embed(minute_size, d_model)
        self.hour_embed = Embed(hour_size, d_model)
        self.weekday_embed = Embed(weekday_size, d_model)
        self.day_embed = Embed(day_size, d_model)
        self.month_embed = Embed(month_size, d_model)

    def forward(self, x):
        x = x.long()
        minute_x = self.minute_embed(x[:, :, 4]) if hasattr(
            self, 'minute_embed') else 0.
        hour_x = self.hour_embed(x[:, :, 3])
        weekday_x = self.weekday_embed(x[:, :, 2])
        day_x = self.day_embed(x[:, :, 1])
        month_x = self.month_embed(x[:, :, 0])

        return hour_x + weekday_x + day_x + month_x + minute_x

In [6]:
class TimeFeatureEmbedding(nn.Module):
    def __init__(self, d_model, embed_type='timeF', freq='h'):
        super(TimeFeatureEmbedding, self).__init__()

        freq_map = {'h': 4, 't': 5, 's': 6,
                    'm': 1, 'a': 1, 'w': 2, 'd': 3, 'b': 3}
        d_inp = freq_map[freq]
        self.embed = nn.Linear(d_inp, d_model, bias=False)

    def forward(self, x):
        import ipdb; ipdb.set_trace()
        return self.embed(x)

In [12]:
# 创建时间序列数据
time_data = torch.arange(0, 24*7)  # 一周的小时数
time_data = time_data.unsqueeze(1)  # 添加 batch 维度

In [13]:
# 创建固定嵌入层和时间嵌入层
fixed_emb = FixedEmbedding(c_in=24*7, d_model=3)
temporal_emb = TemporalEmbedding(d_model=3, embed_type='fixed', freq='h')

RuntimeError: The expanded size of the tensor (1) must match the existing size (2) at non-singleton dimension 1.  Target sizes: [168, 1].  Tensor sizes: [168, 2]

In [ ]:
# 计算固定嵌入和时间嵌入
fixed_embedding = fixed_emb(time_data)
temporal_embedding = temporal_emb(time_data)

# 将嵌入向量转换为 numpy 数组
fixed_embedding_np = fixed_embedding.detach().numpy()
temporal_embedding_np = temporal_embedding.detach().numpy()

# 绘制固定嵌入向量的三维散点图
fig = plt.figure()
ax = fig.add_subplot(121, projection='3d')
ax.scatter(fixed_embedding_np[:, 0], fixed_embedding_np[:, 1], fixed_embedding_np[:, 2], c='r', marker='o')
ax.set_title('Fixed Embedding')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

# 绘制时间嵌入向量的三维散点图
ax = fig.add_subplot(122, projection='3d')
ax.scatter(temporal_embedding_np[:, 0], temporal_embedding_np[:, 1], temporal_embedding_np[:, 2], c='b', marker='o')
ax.set_title('Temporal Embedding')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

plt.show()

#这个演示程序会生成两个三维散点图，一个是固定嵌入层生成的嵌入向量，另一个是时间嵌入层生成的嵌入向量。你可以观察到，固定嵌入层生成的向量是围绕着某个中心点分布的，而时间嵌入层生成的向量则显示出明显的周期性和规律性，这反映了时间信息的编码效果。
